grade: 4/6

How do you know how many intervals in the simpsons/trapezoid method are sufficient to get the desired accuracy?  Do the number of intervals used (100) allow you to reach a similar accuracy to that achieved with the romberg method?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def func(x):
    a = np.e
    b = -2
    c = 10
    return (a**(b*x))*np.cos(c*x)

In [ ]:
def func_integral(x):
    a = np.e
    b = -2
    c = 10
    d = (-1/52)
    e = 5
    return d*(a**(b*x))*(np.cos(c*x)-e*np.sin(c*x))

# Trapezoid

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h)+f(x))

In [ ]:
def trapezoid_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evealuations to take
    #note the number of chunks will be N-1 so if N is odd, then we dont need to adjust the last segment
    
    #define x values to perform trapezoid method
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        
    print("number of iterations:", i)
        
    return Fint

# Simpsons

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3

In [ ]:
def simpsons_method(f,a,b,N):
    
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    Fint = 0.0
    
    #if N is odd or even, wehave different numbers of chunks to add
    if((N%2) == 0):
        lim = 3
    else:
        lim = 2
        
    #perform the integral using simpsons method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        print(i, i+2, x[i], x[i]+h, x[i]+2*h, x[-2], x[-1])
        
    #apply simpsons rule over the last interval if N is even
    if((N%2)==0):
        Fint += simpson_core(f,x[-2], 0.5*h)
        
    print("number of iterations:", i)
    return Fint

# Romberg

In [ ]:
def romberg_core(f,a,b,i):
    
    #we need need the difference b-a
    h = b-a
    
    #and the increment between new func evals
    dh = h/2.**(i)
    
    #we need the cofactor
    K = h/2.**(i+1)
    
    #and the funcion evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i = 0
    
    #define a maximum number of iterations
    imax = 1000
    
    #define an error esimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set and array of integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find the romberg integration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute the new fractional error estimate
        delta = np.fabs( (I[i]-I[i-1])/I[i] )
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i+=1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print("max number of iterations reached.")
                raise StopIteration('stopping iterations after:', i)
                
    #return the answer
    print("number of iterations", i)
    return I[i]

In [ ]:
Answer = func_integral(np.pi)-func_integral(0)
print(Answer)
print("TRAPEZOID")
print(trapezoid_method(func,0,np.pi,100))
print("SIMPSON'S METHOD")
print(simpsons_method(func,0,np.pi,100))
print("ROMBERG")
tolerance = 1.0e-4
RI = romberg_integration(func,0,np.pi,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)